In [ ]:
###########################################################
# MSDS 442: AI Agent Design and Development
# Spring '25
# Dr. Bader
#
# Assignment 4 - Northwestern Memorial – Healthcare Agent
# 
# Kevin Geidel
#
###########################################################

# OBJECTIVE:
#   The following will construct multiple AI agents using the LangChain & LangGraph frameworks. 
#   The agents will represent different departments of Northwestern Memorial Hospital.
#   They will coordinate, synchronize, and act to answer patients'/visitors' questions.

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Python native imports
import os, textwrap

# 3rd party package imports
from IPython.display import display, Image
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Assign experiment-wide variables
model_name = 'gpt-4o-mini'
data_dir = os.path.join('reports', 'Assignment_4')

In [ ]:
# Requirement 1: Define the structure of agent state for the LangGraph
class InquiryState(TypedDict):
    inquiry: str
    next_node: str
    response: str  

In [32]:
# Establish the AI client
llm = ChatOpenAI(model=model_name, temperature=0) 

In [ ]:
def operator_router(state):    
    query = f"""Classify the user's intents based on the following input: '{state['inquiry']}'. 
            List of possible intent values: Greeting, GeneralInquiry, ER, Radiology, PrimaryCare, Cardiology, Pediatrics, BillingInsurance
            Return only the intent value of the inquiry identified with no extra text or characters"""
    
    human_message = HumanMessage(
        content=[
            {"type": "text", "text": query},
        ],
    )

    system_message = SystemMessage(content="You are a helpful assistant tasked with classifying the intent of user's inquiry")
    
    response = llm.invoke([system_message]+[human_message])
    intent = response.content.strip()
            
    response_lower = intent.lower()
    
    if "greeting" in response_lower:
        response = "Hello there, This is Northwestern Memorial Hospital, How can I assisst you today?"
        next_node = END
    elif "generalinquiry" in response_lower:
        response = "For general informtion about nearby parking, hotels and restaurants, please visit https://www.nm.org/ and navigate to Patients & Visitors link "
        next_node = END
    else:
        response = None
        next_node = intent

    return {
        "inquiry": state["inquiry"],
        "next_node": next_node,
        "response": response
    }


In [ ]:
def er_agent(state):
    print("\n\n ER KNOWLEDGE-BASE IS EMPTY \n\n ")
    return {"inquiry": state["inquiry"], "next_node": END, "response": "ER: YOU NEED TO ADD-YOUR-KNOWLEDGE-BASE"}

In [ ]:
def radiology_agent(state):
    print("\n\n Radiology KNOWLEDGE-BASE IS EMPTY \n\n ")
    return {"inquiry": state["inquiry"], "next_node": END, "response": "Radiology: YOU NEED TO ADD-YOUR-KNOWLEDGE-BASE"}

In [ ]:
def primary_care_agent(state):
    print("\n\n Primary Care KNOWLEDGE-BASE IS EMPTY \n\n ")
    return {"inquiry": state["inquiry"], "next_node": END, "response": "Primary Care: YOU NEED TO ADD-YOUR-KNOWLEDGE-BASE"}

In [33]:
def cardiology_agent(state):

    knowledge_base = """
        "inquiry": "Do you have any available appointments with a cardiologist next week?",
         "response": "Appointment availability varies. New patients typically need a referral. Provide the exact date you are looking for so we can check for availability",
         
        "inquiry": "What tests are done during a heart check-up?",
         "response": "Standard tests include EKG, blood pressure, cholesterol screening, and physical exam. Additional tests ordered as needed.",
         
        "inquiry": "How should I prepare for a stress test?",
         "response": "Wear comfortable clothes and walking shoes. Avoid caffeine and heavy meals before the test. Bring a list of medications.",
         
        "inquiry": "What do you recommend to watch for to see if I have signs of heart problems?",
         "response": "Watch for chest pain, shortness of breath, irregular heartbeat, fatigue, and swelling in legs. Go to ER for severe symptoms."},

        "inquiry": "Do you offer heart screenings?",
         "response": "Yes, we provide preventive screenings including calcium scoring, cholesterol tests, and blood pressure monitoring.",
 
        """    
    query = f"""Provide an answer for following user's inquiry: '{state['inquiry']}' using the knowledge_base"""
    
    human_message = HumanMessage(
        content=[
            {"type": "text", "text": query},
        ],
    )

    system_message = SystemMessage(content=f"You are a helpful assistant tasked with answering user's inquiry based on the answers you have in this knowledge_base only: {knowledge_base}")
    
    response = llm.invoke([system_message]+[human_message])
    formatted_response = "Cardiology:: " + response.content.strip()
    
    
    return {"input": state["inquiry"], "next_node": END, "response": formatted_response}



In [ ]:
def pediatrics_agent(state):
    print("\n\n Pediatrics KNOWLEDGE-BASE IS EMPTY \n\n ")
    return {"input": state["inquiry"], "next_node": END, "response": "Pediatrics: YOU NEED TO ADD-YOUR-KNOWLEDGE-BASE."}


In [ ]:
def billing_agent(state):
    print("\n\n BillingInsurance KNOWLEDGE-BASE IS EMPTY \n\n ")
    return {"input": state["inquiry"], "next_node": END, "response": "BillingInsurance: YOU NEED TO ADD-YOUR-KNOWLEDGE-BASE"}
    

In [ ]:

builder = StateGraph(InquiryState)

builder.add_node("Operator", operator_router)
builder.add_node("ER", er_agent)
builder.add_node("Radiology", radiology_agent)
builder.add_node("PrimaryCare", primary_care_agent)
builder.add_node("Cardiology", cardiology_agent)
builder.add_node("Pediatrics", pediatrics_agent)
builder.add_node("BillingInsurance", billing_agent)

builder.set_entry_point("Operator")

builder.add_conditional_edges(
    "Operator",
    lambda x: x["next_node"],
    {
        "ER": "ER",
        "PrimaryCare": "PrimaryCare",
        "Pediatrics": "Pediatrics",
        "Radiology": "Radiology",
        "Cardiology": "Cardiology",
        "BillingInsurance": "BillingInsurance",
        END: END
    }
)

for node in ["ER", "Radiology", "PrimaryCare", "Cardiology", "Pediatrics", "BillingInsurance"]:
    builder.add_edge(node, END)


graph = builder.compile()


In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
def start_chat(wrap_length=100):
    while True:
        user_input = input("User: ")
        print(f"\nUser:\n\n {user_input}")
        if user_input.lower() in {"q", "quit"}:
            print("Goodbye!")
            break
        result = graph.invoke({"inquiry": user_input})
        
        response = result.get("response", "No Response Returned")
        print(f"\n\nResponse:\n\n {textwrap.fill(response, width=wrap_length)} \n\n")

In [ ]:
start_chat()

In [31]:
graph.state

AttributeError: 'CompiledStateGraph' object has no attribute 'state'